In [5]:
import numpy as np
import pandas as pd
import pywt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [6]:
# 1. 데이터 로드
data = pd.read_csv('./data/000660.csv', encoding='cp949')
data = data[int(len(data['종가']) / 2):]
data = data[['종가','거래량']]
data.rename(columns={'종가':'Close','거래량':'Volume'},inplace=True)
prices = data['Close'].values  # 종가 데이터
volumes = data['Volume'].values  # 거래량 데이터

In [19]:
volumes.shape

(93420,)

In [12]:
# 2. CWT 특징 추출
def extract_cwt_features(data, wavelet='morl', scales=np.arange(1, 128)):
    coeffs, _ = pywt.cwt(data, scales, wavelet)
    return coeffs

In [14]:
# 종가와 거래량의 CWT 특징 추출
cwt_price_features = extract_cwt_features(prices)
cwt_volume_features = extract_cwt_features(volumes)

In [40]:
cwt_price_features[:, :1440]

(127, 1440)

In [21]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [22]:
# GPU 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [23]:
# 데이터 전처리 함수
def create_dataset(features, target, time_step=1):
    X, y = [], []
    for i in range(len(features) - time_step):
        if i + time_step < len(target):
            X.append(features[i:(i + time_step), :])
            y.append(target[i + time_step])
    return np.array(X), np.array(y)

In [24]:
# # 예시 데이터 생성 (여기에 실제 데이터를 넣어야 합니다)
# cwt_price_features = np.random.rand(100, 10)  # 예시 데이터
# cwt_volume_features = np.random.rand(100, 10)  # 예시 데이터
# prices = np.random.rand(101)  # 예시 가격 데이터
# volumes = np.random.rand(101)  # 예시 거래량 데이터

# 특징과 타겟 변수를 생성
X_price, y_price = create_dataset(cwt_price_features.T, prices[1:], time_step=10)
X_volume, y_volume = create_dataset(cwt_volume_features.T, volumes[1:], time_step=10)

In [28]:
X_price.shape, X_volume.shape

((93409, 10, 127), (93409, 10, 127))

In [32]:
# X를 합치기
X = np.concatenate((X_price, X_volume), axis=2)

In [35]:
# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_price, test_size=0.2, random_state=42)

c:\ProgramData\anaconda3\envs\final\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 26906904576.0000
Epoch 2/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 27013210112.0000
Epoch 3/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 26859464704.0000
Epoch 4/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 26824628224.0000
Epoch 5/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 26779260928.0000
Epoch 6/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 26817726464.0000
Epoch 7/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 26766004224.0000
Epoch 8/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 26694193152.0000
Epoch 9/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 26796509184.0000
Epoch 10/10
2336/2336 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 26622965760.0000
584/584 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 26522447872.0000
테스트 손실: 26622119936.0000


In [42]:
y_train.shape

(74727,)

In [ ]:

# 모델 정의
model = tf.keras.Sequential([
    # tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=32)

# 모델 평가
loss = model.evaluate(X_test, y_test)
print(f'테스트 손실: {loss:.4f}')
